# Simulated Data

We study the performance of Hamiltonian Monte Carlo and Stochastic Gradient Hamiltonian Monte Carlo for classification tasks. 



# Binary Classification

First, a 2-dimensional binary classification problem is simulated. Data is generated from two well separated clusters, so a linear classification model is well suited. The generated data is split into train and test datasets. 

In [1]:
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split

D=2
centers = [[-5, 0],  [5, -1]]
X, y = make_blobs(n_samples=1000, centers=centers, cluster_std=1,random_state=40)
X = (X - X.mean(axis=0)) / X.std(axis=0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

A logistic regression model is fitted using the 2-dimensional features and $L_2$ regularization with $\alpha=0.25$. The weights and bias are initialized with $\mathbf w=0$ and $b=0$. Stochastic Gradient descent is also used to estimate the parameters using $10e4$ epochs, a learning rate $\epsilon=1e-5$ and a batch size $b=50$.

In [2]:
try:
    from google.colab import drive
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


In [3]:
if COLAB:
    !git clone https://github.com/sherna90/hamiltonian_montecarlo/
    !pip install cupy

else:
    import sys
    sys.path.append("../") 

In [4]:
import hamiltonian.models.cpu.logistic as base_model_cpu
import hamiltonian.inference.cpu.sgd as inference_cpu
import numpy as np

D=X_train.shape[1]
N=X_train.shape[0]

epochs = 1e4
eta=1e-5
batch_size=50
alpha=1/4.

start_p={'weights':2*np.random.random((D,1)),'bias':2*np.random.random(1)}
hyper_p={'alpha':alpha}

model_cpu=base_model_cpu.logistic(hyper_p)
optim_cpu=inference_cpu.sgd(model_cpu,start_p,step_size=eta)
par_cpu,loss=optim_cpu.fit(epochs=epochs,batch_size=batch_size,gamma=0.9,
                   X_train=X_train,y_train=y_train,verbose=True)
y_pred=model_cpu.predict(par_cpu,X_test,batchsize=batch_size)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

  3%|▎         | 265/10000 [00:00<00:07, 1334.46it/s]

loss: 0.8813


 11%|█         | 1124/10000 [00:00<00:07, 1156.37it/s]

loss: 0.1525


 23%|██▎       | 2265/10000 [00:01<00:05, 1310.80it/s]

...

loss: 0.1527


 92%|█████████▏| 9239/10000 [00:07<00:00, 1384.81it/s]

loss: 0.1527


100%|██████████| 10000/10000 [00:07<00:00, 1300.21it/s]

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       116
           1       1.00      1.00      1.00       134

    accuracy                           1.00       250
   macro avg       1.00      1.00      1.00       250
weighted avg       1.00      1.00      1.00       250

[[116   0]
 [  0 134]]


In [7]:
import cupy as cp

ModuleNotFoundError: No module named 'cupy'

In [5]:
import hamiltonian.models.gpu.logistic as base_model_gpu
import hamiltonian.inference.gpu.sgd as inference_gpu

model_gpu=base_model_gpu.logistic(hyper_p)
optim_gpu=inference_gpu.sgd(model_gpu,start_p,step_size=eta)
par_gpu,loss=optim_cpu.fit(epochs=epochs,batch_size=batch_size,gamma=0.9,
                   X_train=X_train,y_train=y_train,verbose=True)
y_pred=model_gpu.predict(par_gpu,X_test,batchsize=batch_size)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

ModuleNotFoundError: No module named 'cupy'

# Hamiltonian Monte Carlo

Now we want to estimate posterior distributions for the logistic regression model. We run $100$ burn-in samples and then we sample $1e3$ iterations of the Hamiltonian Monte Carlo algorithm. 

In [ ]:
import hamiltonian.inference.cpu.hmc as sampler_gpu

burnin=1e2

hmc_gpu=sampler_gpu.hmc(model_cpu,start_p,path_length=1,step_size=eta)
samples,loss,positions,momentums=hmc_gpu.sample(1e3,burnin,None,
                                            X_train=X_train,y_train=y_train)




  0%|          | 0/100 [00:00<?, ?it/s]

  1%|          | 1/100 [00:22<36:38, 22.20s/it]

loss: 0.1615




  2%|▏         | 2/100 [00:40<34:20, 21.02s/it]

  3%|▎         | 3/100 [00:42<24:56, 15.43s/it]

  4%|▍         | 4/100 [00:49<20:41, 12.93s/it]

  5%|▌         | 5/100 [00:57<17:58, 11.35s/it]

  6%|▌         | 6/100 [01:14<20:17, 12.95s/it]

  7%|▋         | 7/100 [01:33<23:01, 14.86s/it]

  8%|▊         | 8/100 [01:49<23:11, 15.13s/it]

  9%|▉         | 9/100 [02:09<25:17, 16.67s/it]

 10%|█         | 10/100 [02:13<19:04, 12.71s/it]

 11%|█         | 11/100 [02:24<18:27, 12.44s/it]

loss: 0.1615




 12%|█▏        | 12/100 [02:36<17:40, 12.05s/it]

 13%|█▎        | 13/100 [02:36<12:25,  8.57s/it]

 14%|█▍        | 14/100 [02:50<14:40, 10.24s/it]

 15%|█▌        | 15/100 [03:12<19:14, 13.59s/it]

 16%|█▌        | 16/100 [03:26<19:15, 13.76s/it]

 17%|█▋        | 17/100 [03:31<15:36, 11.28s/it]

 18%|█▊        | 18/100 [03:45<16:27, 12.05s/it]

 19%|█▉        | 19/100 [04:01<17:55, 13.27s/it]

 20%|██        | 20/100 [04:22<20:33, 15.41s/it]

 21%|██        | 21/100 [04:41<21:54, 16.64s/it]

...



 70%|███████   | 702/1000 [2:13:29<52:44, 10.62s/it]  

 70%|███████   | 703/1000 [2:13:39<52:21, 10.58s/it]

 70%|███████   | 704/1000 [2:13:58<1:03:38, 12.90s/it]

 70%|███████   | 705/1000 [2:14:04<53:27, 10.87s/it]  

 71%|███████   | 706/1000 [2:14:09<45:21,  9.26s/it]

 71%|███████   | 707/1000 [2:14:16<41:53,  8.58s/it]

 71%|███████   | 708/1000 [2:14:19<33:25,  6.87s/it]

 71%|███████   | 709/1000 [2:14:38<50:35, 10.43s/it]

 71%|███████   | 710/1000 [2:14:57<1:02:46, 12.99s/it]

 71%|███████   | 711/1000 [2:15:04<53:39, 11.14s/it]  

 71%|███████   | 712/1000 [2:15:07<42:27,  8.85s/it]

 71%|███████▏  | 713/1000 [2:15:17<43:35,  9.11s/it]

 71%|███████▏  | 714/1000 [2:15:25<41:23,  8.68s/it]

 72%|███████▏  | 715/1000 [2:15:37<47:01,  9.90s/it]

 72%|███████▏  | 716/1000 [2:15:45<43:23,  9.17s/it]

 72%|███████▏  | 717/1000 [2:16:01<53:30, 11.35s/it]

 72%|███████▏  | 718/1000 [2:16:02<37:46,  8.04s/it]

 72%|███████▏  | 719/1000 [2:16:14<43:17,  9.24s/it]

 72%|███████▏  |

loss: 0.1619




 80%|████████  | 802/1000 [2:30:07<29:32,  8.95s/it]

 80%|████████  | 803/1000 [2:30:16<29:18,  8.93s/it]

 80%|████████  | 804/1000 [2:30:23<26:53,  8.23s/it]

 80%|████████  | 805/1000 [2:30:42<37:59, 11.69s/it]

 81%|████████  | 806/1000 [2:31:03<46:30, 14.39s/it]

 81%|████████  | 807/1000 [2:31:07<36:11, 11.25s/it]

 81%|████████  | 808/1000 [2:31:16<33:40, 10.52s/it]

 81%|████████  | 809/1000 [2:31:26<32:59, 10.37s/it]

 81%|████████  | 810/1000 [2:31:44<40:23, 12.75s/it]

 81%|████████  | 811/1000 [2:32:04<46:36, 14.79s/it]

 81%|████████  | 812/1000 [2:32:16<44:01, 14.05s/it]

 81%|████████▏ | 813/1000 [2:32:28<41:36, 13.35s/it]

 81%|████████▏ | 814/1000 [2:32:49<48:44, 15.72s/it]

 82%|████████▏ | 815/1000 [2:33:04<47:33, 15.42s/it]

 82%|████████▏ | 816/1000 [2:33:19<47:23, 15.46s/it]

 82%|████████▏ | 817/1000 [2:33:36<48:30, 15.90s/it]

 82%|████████▏ | 818/1000 [2:33:52<47:59, 15.82s/it]

 82%|████████▏ | 819/1000 [2:34:04<44:00, 14.59s/it]

 82%|████████▏ | 820/1000 

loss: 0.1619




 90%|█████████ | 902/1000 [2:48:07<13:29,  8.26s/it]

 90%|█████████ | 903/1000 [2:48:07<09:37,  5.95s/it]

 90%|█████████ | 904/1000 [2:48:11<08:27,  5.28s/it]

 90%|█████████ | 905/1000 [2:48:18<09:02,  5.72s/it]

 91%|█████████ | 906/1000 [2:48:40<16:41, 10.66s/it]

 91%|█████████ | 907/1000 [2:48:45<13:51,  8.94s/it]

 91%|█████████ | 908/1000 [2:48:52<12:54,  8.41s/it]

 91%|█████████ | 909/1000 [2:49:15<19:38, 12.95s/it]

 91%|█████████ | 910/1000 [2:49:28<19:14, 12.83s/it]

 91%|█████████ | 911/1000 [2:49:51<23:29, 15.83s/it]

In [ ]:
par_mean={var:np.mean(samples[var],axis=0).reshape(start_p[var].shape) for var in samples.keys()}
par_var={var:np.var(samples[var],axis=0).reshape(start_p[var].shape) for var in samples.keys()}
y_pred=model_cpu.predict(X_test,par_mean)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))




In [ ]:
import matplotlib.pyplot as plt

def plot_uncertainty_hyperplane(par_mean,par_var, color):
    bd = lambda x0,par :  (-(x0 * par['weights'][0]) - par['bias']) / par['weights'][1]
    r=np.linspace(xmin,xmax)
    par_m={var:par_mean[var]-np.sqrt(par_var[var]) for var in par_mean.keys()}
    par_p={var:par_mean[var]+np.sqrt(par_var[var]) for var in par_mean.keys()}
    plt.plot(r,bd(r,par_mean),ls="-", color=color)
    plt.plot(r,bd(r,par_m),ls="--", color=color, alpha=0.5)
    plt.plot(r,bd(r,par_p),ls="--", color=color, alpha=0.5)
    #plt.fill_between(r, bd(r,par_mean),  bd(r,par_p), color=color, alpha=0.5)
    #plt.fill_between(r, bd(r,par_m),  bd(r,par_mean), color=color, alpha=0.5)
    
plt.figure()
plt.scatter(X[:, 0], X[:, 1],marker='o', c=y,s=25, edgecolor='k')
plt.axis('tight')
xmin, xmax = plt.xlim()
ymin, ymax = plt.ylim()
plot_uncertainty_hyperplane(par_mean, par_var,"b")
plt.show()